In [ ]:
! pip install beautifulsoup4

In [ ]:
! pip install httplib2

In [ ]:
! pip install requests

In [ ]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('scripts/')]

onlyfiles

In [ ]:
movies = [item.split('_')[1].replace('.txt', '').replace('-', '_') for item in onlyfiles]
movies

In [ ]:
pd.DataFrame(list(set(movies))).to_csv('movies_all.csv')

In [ ]:
import requests
import httplib2
from bs4 import BeautifulSoup, SoupStrainer

movie_tomdata = []
http = httplib2.Http()
for movie in movies:
    url = 'https://www.rottentomatoes.com/m/%s' % movie
    status, response = http.request(url)

    soup = BeautifulSoup(response, "html.parser")

    for tag in soup.select("score-board"):
        aud = tag['audiencescore']
        tom = tag['tomatometerscore']
        print(movie, aud, tom)
        movie_tomdata.append((movie, aud, tom))

In [ ]:
len(movie_tomdata), len(set(movie_tomdata))

In [ ]:
! pip install pandas

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(list(set(movie_tomdata))).to_csv('rotten_tomatoes.csv')

In [ ]:
# function to extract all text given a URL
def get_text(page_url):
    res = requests.get(page_url)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    text = soup.find_all(text=True)
    return text
#     script_text = '\n'.join([t for t in text if t.parent.name == 'pre'])
#     return script_text

In [ ]:
get_text('https://www.rottentomatoes.com/m/127_hours')

In [ ]:
# for each genre page, get all the script links and extract the corresponding
# script text and save it to a file with the format 'genre_moviename.txt'

for link in genre_links:
    genre = link.split('gen-')[1].split(',')[0].split('.')[0]
    http = httplib2.Http()
    status, response = http.request(link)
    for link in BeautifulSoup(response, 'html.parser', parse_only=SoupStrainer('a')):
        if link.has_attr('href'):
            if '/movie-scripts/scripts/' in link['href']:
                script_link = 'https://www.actorpoint.com' + link['href']
                movie_name = script_link.split('/')[-1].split('.')[0]
                text = get_text(script_link)
                filename = 'scripts/' + genre + '_' + movie_name + '.txt'
                print(filename)
                with open(filename, 'w') as f:
                    f.write(text)
